# Preparing enviroment

In [2]:
!pip -q install opendatasets
!pip -q install pandas
!pip -q install darknet
!pip -q install numba

In [4]:
import opendatasets as od
import pandas

od.download(
    "https://mm.cs.uec.ac.jp/uecfoodpix/UECFOODPIXCOMPLETE.tar")

Using downloaded and verified file: ./UECFOODPIXCOMPLETE.tar


In [5]:
! tar xf /kaggle/working/UECFOODPIXCOMPLETE.tar
! rm /kaggle/working/UECFOODPIXCOMPLETE.tar

# importes

In [88]:
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import pydot
import darknet as dn
import os
import cv2
import numpy as np
from glob import glob
from scipy.io import loadmat
from tensorflow import keras
from keras import layers
from os import path
import warnings
warnings.filterwarnings('ignore')


# Directories

In [7]:
DRIVE_FOLDER = "/kaggle/working/UECFOODPIXCOMPLETE/data/UECFoodPIXCOMPLETE"
SUBSET_FOLDER = ""

TRAIN_IMAGE_DIR = pathlib.Path(DRIVE_FOLDER + SUBSET_FOLDER + "/train/img")
TRAIN_MASK_DIR = pathlib.Path(DRIVE_FOLDER + SUBSET_FOLDER + "/train/mask")

TEST_IMAGE_DIR = pathlib.Path(DRIVE_FOLDER + SUBSET_FOLDER + "/test/img")
TEST_MASK_DIR = pathlib.Path(DRIVE_FOLDER + SUBSET_FOLDER + "/test/mask")

# Settings

In [8]:
IMAGE_SIZE = 512
BATCH_SIZE = 4

CLASSES = 102

NUM_TRAIN_IMAGES = 6300
NUM_VAL_IMAGES = 2700

# Loading data

In [9]:
train_images = sorted(glob(str(TRAIN_IMAGE_DIR/'*.jpg')))[:NUM_TRAIN_IMAGES]
train_masks = sorted(glob(str(TRAIN_MASK_DIR/'*.png')))[:NUM_TRAIN_IMAGES]

val_images = sorted(glob(str(TRAIN_IMAGE_DIR/'*.jpg')))[
    NUM_TRAIN_IMAGES : NUM_VAL_IMAGES + NUM_TRAIN_IMAGES
]
val_masks = sorted(glob(str(TRAIN_MASK_DIR/'*.png')))[
    NUM_TRAIN_IMAGES : NUM_VAL_IMAGES + NUM_TRAIN_IMAGES
]
print(train_images[0])
print(train_masks[0])
print(val_images[0])
print(val_masks[0])

/kaggle/working/UECFOODPIXCOMPLETE/data/UECFoodPIXCOMPLETE/train/img/1.jpg
/kaggle/working/UECFOODPIXCOMPLETE/data/UECFoodPIXCOMPLETE/train/mask/1.png
/kaggle/working/UECFOODPIXCOMPLETE/data/UECFoodPIXCOMPLETE/train/img/5681.jpg
/kaggle/working/UECFOODPIXCOMPLETE/data/UECFoodPIXCOMPLETE/train/mask/5681.png


# Create **image** **,** **label** tuples

In [75]:
def read_image(image_path, mask=False):
    image = tf.io.read_file(image_path)
    if mask:
        image = tf.image.decode_png(image, channels=1)
        red_channel = tf.expand_dims(image[:,:,0], axis=-1)
        red_channel.set_shape([None, None, 1])
        image = tf.image.resize(red_channel, [IMAGE_SIZE, IMAGE_SIZE], method='nearest')
    else:
        image = tf.image.decode_png(image, channels=3)
        image.set_shape([None, None, 3])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
        image = tf.keras.applications.resnet50.preprocess_input(image)
    return image

In [63]:
def load_data(image_list, mask_list):
    image = read_image(image_list)
    mask = read_image(mask_list, mask=True)
    return image, mask

In [64]:
def data_generator(image_list, mask_list):
    with tf.device("CPU"):
      dataset = tf.data.Dataset.from_tensor_slices((image_list, mask_list))
      dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
      dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset

In [76]:
train_dataset = data_generator(train_images, train_masks)
val_dataset = data_generator(val_images, val_masks)

print("Train Dataset:", train_dataset)
print("Val Dataset:", val_dataset)

Train Dataset: <BatchDataset shapes: ((4, 512, 512, 3), (4, 512, 512, 1)), types: (tf.float32, tf.uint8)>
Val Dataset: <BatchDataset shapes: ((4, 512, 512, 3), (4, 512, 512, 1)), types: (tf.float32, tf.uint8)>


# Vizualization callback

In [67]:
# Visualization callback to see how mask prediction progresses.
class MaskCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        plot_mask_predictions(val_dataset, 3)

def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

In [68]:
def plot_mask_predictions(dataset, examples=3):
    plt.figure(figsize=(21, 7 * examples))

    for i, (image, mask) in enumerate(dataset.take(examples)):
        ax = plt.subplot(examples, 3, 3*i+1)
        plt.imshow(image[0].numpy().astype("uint8"))
        plt.axis("off")

        ax = plt.subplot(examples, 3, 3*i+2)
        plt.imshow(mask[0].numpy().astype("uint8").squeeze(axis=2), cmap='gray', vmin=0, vmax=tf.math.reduce_max(mask[0]))
        plt.axis("off")


        pred_mask = create_mask(model.predict(image))

        ax = plt.subplot(examples, 3, 3*i+3)
        plt.imshow(pred_mask.numpy().astype("uint8").squeeze(axis=2), cmap='gray', vmin=0, vmax=tf.math.reduce_max(pred_mask))
        plt.axis("off")

    plt.show()

In [69]:
class IOUAccuracyCallback(tf.keras.callbacks.Callback):
    def __init__(self, validation_data):
        super(IOUAccuracyCallback, self).__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}

        # Get the validation data and model predictions
        x_val, y_val = self.validation_data
        y_pred = self.model.predict(x_val)

        # Compute the IOU accuracy
        iou = self.compute_iou(y_val, y_pred)

        # Add the IOU accuracy to the logs dictionary
        logs['iou'] = iou

    def compute_iou(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.bool)
        y_pred = tf.cast(y_pred >= 0.5, tf.bool)

        intersection = tf.logical_and(y_true, y_pred)
        union = tf.logical_or(y_true, y_pred)

        iou = tf.reduce_mean(tf.cast(tf.reduce_sum(intersection, axis=[1,2,3]) /
                                     tf.reduce_sum(union, axis=[1,2,3]), tf.float32))

        return iou

# Model

In [70]:
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

In [71]:
def DeeplabV3Plus(image_size, num_classes):
    model_input = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    resnet50 = keras.applications.ResNet50(
        weights="imagenet", include_top=False, input_tensor=model_input
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same")(x)
    return keras.Model(inputs=model_input, outputs=model_output)


model = DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=CLASSES)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 518, 518, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 256, 256, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 256, 256, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [89]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=loss,
    metrics=["accuracy"],
)

## Model Training

In [91]:
EPOCHS = 10

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="cp-{epoch:04d}.h5", 
    verbose=1, 
    save_weights_only=True , monitor='val_loss', save_best_only=True)

early_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=5,verbose=1,mode='auto')

iou_callback = IOUAccuracyCallback(validation_data=val_dataset)

In [ ]:
tf.get_logger().setLevel('INFO')
model_history = model.fit(
    train_dataset, 
    epochs=EPOCHS, 
    validation_data=val_dataset, 
    callbacks=[MaskCallback(), cp_callback, iou_callback, early_callback]
)

Epoch 1/10


Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...


   1/1575 [..............................] - ETA: 2:14:58 - loss: 0.8035 - accuracy: 0.8054

Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...


   2/1575 [..............................] - ETA: 11:56 - loss: 0.8445 - accuracy: 0.7588  

...

Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...


 392/1575 [======>.......................] - ETA: 9:06 - loss: 1.2986 - accuracy: 0.6599

Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...


 393/1575 [======>.......................] - ETA: 9:05 - loss: 1.2967 - accuracy: 0.6604

Cleanup called...
Cleanup called...
Cleanup called...
Cleanup called...


In [ ]:
plt.plot(model_history.history["loss"])
plt.title("Training Loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.show()

plt.plot(model_history.history["accuracy"])
plt.title("Training Accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.show()

plt.plot(model_history.history["val_loss"])
plt.title("Validation Loss")
plt.ylabel("val_loss")
plt.xlabel("epoch")
plt.show()

plt.plot(model_history.history["val_accuracy"])
plt.title("Validation Accuracy")
plt.ylabel("val_accuracy")
plt.xlabel("epoch")
plt.show()

In [ ]:
# Loading the Colormap
colormap = loadmat(
    "./instance-level_human_parsing/instance-level_human_parsing/human_colormap.mat"
)["colormap"]
colormap = colormap * 100
colormap = colormap.astype(np.uint8)


def infer(model, image_tensor):
    predictions = model.predict(np.expand_dims((image_tensor), axis=0))
    predictions = np.squeeze(predictions)
    predictions = np.argmax(predictions, axis=2)
    return predictions


def decode_segmentation_masks(mask, colormap, n_classes):
    r = np.zeros_like(mask).astype(np.uint8)
    g = np.zeros_like(mask).astype(np.uint8)
    b = np.zeros_like(mask).astype(np.uint8)
    for l in range(0, n_classes):
        idx = mask == l
        r[idx] = colormap[l, 0]
        g[idx] = colormap[l, 1]
        b[idx] = colormap[l, 2]
    rgb = np.stack([r, g, b], axis=2)
    return rgb


def get_overlay(image, colored_mask):
    image = tf.keras.preprocessing.image.array_to_img(image)
    image = np.array(image).astype(np.uint8)
    overlay = cv2.addWeighted(image, 0.35, colored_mask, 0.65, 0)
    return overlay


def plot_samples_matplotlib(display_list, figsize=(5, 3)):
    _, axes = plt.subplots(nrows=1, ncols=len(display_list), figsize=figsize)
    for i in range(len(display_list)):
        if display_list[i].shape[-1] == 3:
            axes[i].imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        else:
            axes[i].imshow(display_list[i])
    plt.show()


def plot_predictions(images_list, colormap, model):
    for image_file in images_list:
        image_tensor = read_image(image_file)
        prediction_mask = infer(image_tensor=image_tensor, model=model)
        prediction_colormap = decode_segmentation_masks(prediction_mask, colormap, 20)
        overlay = get_overlay(image_tensor, prediction_colormap)
        plot_samples_matplotlib(
            [image_tensor, overlay, prediction_colormap], figsize=(18, 14)
        )